# Dialogue system using Large Language Model(LLM)

This is the recipe for experiencing dialogue system using large language model.<br>
Here we use Phi-3.5, that is invented by Microsoft and Whisper, that is invented by OpenAI.

こちらは大規模言語モデル(LLM)を用いた対話システムをお試しするためのレシピです。<br>
こちらではMicrosoftによって開発されたPhi-3.5および、OpenAIによって開発されたWhisperを用います。


Reference/参考リンク:<br>
https://huggingface.co/microsoft/Phi-3.5-mini-instruct
https://huggingface.co/openai/whisper-large-v3-turbo


## Installing required libraries
Here we install the required libraries.

こちらでは必要なライブラリをインストールします。

In [ ]:
!pip install transformers torchaudio datasets

## Import necessary libraries and define variable
Here we import necessary libraries and define variable.

こちらでは必要なライブラリをインポートし、変数を定義しています。

In [ ]:
import torch
import torchaudio
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSpeechSeq2Seq, AutoProcessor

audio_file = "recorded_audio.wav"
print(torch.cuda.is_available())

## Set your favorite language

Please set the language that you want to use on this dialogue system.<br>
For example, you can use English, Japanese, Chinese, Thai, and so on...

対話システムで利用したい言語を設定してください。<br>
例えば、英語、日本語、中国語、タイ語などが指定できます。。。

In [ ]:
# Please input your favorite language in lower case.
language = "japanese"

## Load the pre-trained Whisper model for speech recognition
Here we load the pre-trained Whisper for speech recognition.

こちらでは音声認識に必要なWhisperを読み込みます。

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3-turbo"

w_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
w_model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

speech_recognizer = pipeline("automatic-speech-recognition", model=w_model, tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor, torch_dtype=torch_dtype, device=device)

## Load a pre-trained LLM
We load a pre-trained LLM, that is Phi-3.5.

学習済みLLMであるPhi-3.5を読み込みます。

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", device_map="cuda", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

## Record your voice
You can record your voice to say to the dialogue system.<br>
To use this function, you need to permit authority on the dialogue window that demand to use microphone.<br>
Push `録音開始` button to start recording and push `録音停止` button to finish recording.<br>
During recording, the message <font color="red">録音中</font> will be displayed.<br>
You can push `録音開始` and `録音停止` button whenever you want to record new voice.<br>
To be more detailed, it uses JavaScript to record your voice internally.

対話システムに渡すための音声を録音することができます。<br>
この機能を使用されるにあたっては、マイクの利用を許可する旨のダイアログにおいて、権限を許可する必要があります。<br>
`録音開始`ボタンを押すと録音を開始でき、`録音停止`ボタンを押すと録音を終了できます。<br>
録音中は、<font color="red">録音中</font> という文字が表示されます。<br>
新しい音声を録音したいとき、`録音開始`ボタンおよび`録音停止`ボタンは何回でも押せます。<br>
より詳しくお伝えすると、ここでは内部的には音声を録音するためにJavaScriptを使用しています。

In [ ]:
from IPython.display import Javascript, display
from google.colab import output
import base64

# JavaScript: 録音制御および「録音中」表示の更新
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
var mediaRecorder = null;
var audioChunks = [];

function startRecording() {
  navigator.mediaDevices.getUserMedia({ audio: true })
    .then(stream => {
      mediaRecorder = new MediaRecorder(stream, { mimeType: 'audio/webm' });
      mediaRecorder.start();
      audioChunks = [];

      mediaRecorder.addEventListener('dataavailable', event => {
        audioChunks.push(event.data);
      });

      mediaRecorder.addEventListener('stop', () => {
        const audioBlob = new Blob(audioChunks, { type: 'audio/wav' });
        const reader = new FileReader();
        reader.readAsDataURL(audioBlob);  // Base64に変換

        reader.onloadend = () => {
          const base64String = reader.result.split(',')[1];  // データ部分のみ抽出
          google.colab.kernel.invokeFunction('notebook.save_audio', [base64String], {});
        };
      });

      // 録音中のステータスを表示
      document.getElementById('status').textContent = '録音中...';
    });
}

function stopRecording() {
  if (mediaRecorder) {
    mediaRecorder.stop();
    // 録音中のステータスを消去
    document.getElementById('status').textContent = '';
  }
}
"""

# JavaScriptコードをColabにロードする
display(Javascript(RECORD_JS))

# Python: 録音結果を処理し保存する
def save_audio(base64_string):
    # Reference: https://qiita.com/hiroyuki_kawahara/items/6a221f33d9c5078c3036
    audio_bytes = base64.b64decode(base64_string)  # Base64をデコード

    with open(audio_file, 'wb') as f:
        f.write(audio_bytes)

    print(f"録音した音声を '{audio_file}' に保存しました。")

# JavaScriptからのコールバックを登録
output.register_callback('notebook.save_audio', save_audio)

# 録音用のUIを表示する関数
def display_audio_recorder():
    display(Javascript("""
        const container = document.createElement('div');
        container.style.marginBottom = '10px';

        const startButton = document.createElement('button');
        startButton.textContent = '録音開始';
        startButton.style.marginRight = '10px';
        startButton.onclick = startRecording;

        const stopButton = document.createElement('button');
        stopButton.textContent = '録音停止';
        stopButton.onclick = stopRecording;

        const status = document.createElement('span');
        status.id = 'status';
        status.style.marginLeft = '10px';
        status.style.color = 'red';

        container.appendChild(startButton);
        container.appendChild(stopButton);
        container.appendChild(status);

        document.body.appendChild(container);
    """))

display_audio_recorder()


## Perform speech recognition on a sample audio

Here this cell recognize the voice as a text format.<br>
The result of this recognition is shown on the following of the cell.

ここで、このセルは音声をテキスト形式で認識しています。<br>
認識結果はこのセルの下に表示されます。

In [ ]:
text = speech_recognizer(audio_file, generate_kwargs={"language": language})["text"]
print(f"Recognized Speech: {text}")

## Set what the system looks like
Here please set how the system is.<br>
For example, you can arrange the system like some experts or developer and so on.<br>
Here you should set it in English.

ここでは、システムがどういった性格を持つかを指定してください。<br>
例えば、そのシステムがエキスパートであったり開発者であったりと指定できます。<br>
ここは、英語で指定してください。

In [ ]:
character = "expert for writing Python"

## Use LLM to generate a response based on the recognized text
Here we use LLM to generate a response based on the recognized text.<br>
Did you satisfy with the reply with LLM?

こちらで認識されたテキストについてLLMが返答を生成します。<br>
LLMが生成した結果には満足していただけましたか？

In [ ]:
messages = [
    {"role": "system", "content": "You are {} {}.".format("an" if character[0] in ['a', 'i', 'u', 'e', 'o'] else "a", character)},
    {"role": "user", "content": text},
]
response = generator(messages, max_new_tokens=500, return_full_text=False, temperature=0.0, do_sample=False)
print(f"Generated Response: {response[0]['generated_text']}")